# YOLOv12   trainning notebook


Lo primero a tener  en cuenta es que esto es para Colab.

si quieres hacer el loca, tienes conda en segunda línea y cambias las instalaciones

# 0. Montar Drive

In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Importar carpetas desde Google Drive (si no están en Rar)

In [ ]:
# 2. Definir las carpetas en Google Drive que deseas importar
# Por ejemplo, si tienes dos carpetas: una para imágenes y otra para etiquetas
folders_to_copy = [
    "/content/drive/MyDrive/Fotos_elegidas"  # Carpeta con imágenes
   # "/content/drive/MyDrive/Tmodel_rar_640_tiles_labels/fotos_elegidas/labels"   # Carpeta con archivos txt
]

# 3. Definir un directorio de destino en Colab donde copiarás estas carpetas
destination_base = "/content/dataset_importado"
import os
os.makedirs(destination_base, exist_ok=True)

# 4. Copiar cada carpeta desde Drive al directorio local
for folder in folders_to_copy:
    folder_name = os.path.basename(folder)
    dest = os.path.join(destination_base, folder_name)
    # El comando cp -r copia recursivamente la carpeta
    !cp -r "{folder}" "{dest}"

# 5. Verificar que se copiaron las carpetas
!ls -l "{destination_base}"

# 2. Copiar el repositorio de YOLOv12 en nuestro Colab


Con esto tenemos acceso a todos los archivos que contienen lo necesario para montar y correr YOLO12

# 2.1. Lo primero es incorporar este archivo que nos indica en el repositorio

In [ ]:
!wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl


# 2.2. Lo segundo es clonar toda la información de Yolo v 12


In [ ]:
# Esto es un ejemplo; en Colab, generalmente no se crea un entorno conda, sino se instala en el entorno actual.
# Si estuvieras en local, podrías hacer:
# conda create -n yolov12 python=3.11
# conda activate yolov12


#como curro en Colab:
!git clone https://github.com/sunsmarterjie/yolov12.git
#!mv flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl yolov12/


# 2.3. Comrpobar que todo está, y con esto a partir de ahora nos colocamos en la carpeta yolov12 del directorio de colab para correr código

In [ ]:
%cd yolov12

/content/yolov12


# 2.4. ATENCIÓN!

Antes de ejecutar siguiente celda, mueve el archivo solo flash attention dentro de la carpeta de yolov12 descargada justo ahora

Esto es por que en el código anterior te has colocado en content/yolov12, y para que todo corra smooth, el primer archivo descargado que aparecerá a la izquierda suelto, debe estar dentro de la carpeta de yolo12

In [ ]:
!pip install -r /content/yolov12/requirements.txt


# 2.5 Volver a carpeta de YOLOV12 e instalar

In [ ]:
%cd /content/yolov12
!pip install -e .


# 3. Importar Ultralytics y nombrar el modelo

In [ ]:
# 0. Ejecutar el entrenamiento de YOLOv8
from ultralytics import YOLO

# 1. Crear el modelo usando una configuración de YOLOv12.
# 'yolov12n.yaml' es el archivo de configuración que define la arquitectura,
# hiperparámetros, y otros detalles específicos para la versión "nano" de YOLOv12.
model = YOLO("yolov12s.yaml")





# 4. Importar el data.yaml con los directorios adecuados

In [ ]:

################################IMPORTAR EL DAYA.YAML UNA VEZ TIENES LA ESTRUCTURA  ####################################################
####################################################################################
# Copiar el archivo data.yaml desde Drive a la ruta deseada en Colab
!cp "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/data.yaml" "/content/dataset_importado_bueno/Fotos_elegidas/data.yaml"

# Verificar que se copió correctamente
!ls -l "/content/dataset"

# 5. Entrenar el modelo

Ajustar parámetros a como tú lo necesites. los de abajo son pasos de augmentation

In [ ]:


# 2. Entrenar el modelo.
results = model.train(
  data='/content/dataset_importado/Fotos_elegidas/data.yaml',   # Archivo de configuración del dataset (por ejemplo, COCO).
  epochs=100,                                             # Número de épocas: el modelo recorrerá todo el dataset 600 veces.
  batch=16,                                                 # Tamaño de lote: 256 imágenes por cada batch.
  imgsz=640,                                              # Tamaño de imagen: se reescala cada imagen a 640x640 píxeles.
  scale=0.5,          # S:0.9; M:0.9; L:0.9; X:0.9        # Factor de escalado en algunas transformaciones de augmentación.
                                                            # (El comentario sugiere valores recomendados para diferentes tamaños: S, M, L, X).
  mosaic=1.0,                                               # Activación del mosaic augmentation (completo si es 1.0).
  mixup=0.0,         # S:0.05; M:0.15; L:0.15; X:0.2       # mixup augmentation está desactivado (0.0); otros valores podrían ser pequeños (por ejemplo, 0.05 para S).
  copy_paste=0.1,     # S:0.15; M:0.4; L:0.5; X:0.6         # Copy-paste augmentation se activa con una probabilidad de 0.1.
  #device="0,1,2,3",                                        # Usar GPUs con IDs 0, 1, 2 y 3 para el entrenamiento.
  resume=True
)

 scale=0.5,  # S:0.9; M:0.9; L:0.9; X:0.9
  mosaic=1.0,
  mixup=0.0,  # S:0.05; M:0.15; L:0.15; X:0.2
  copy_paste=0.1,  # S:0.15; M:0.4; L:0.5; X:0.6





data='coco.yaml': Indica que se usará un archivo de configuración de dataset (por ejemplo, con formato COCO) que define las rutas de las imágenes de entrenamiento, validación, el número de clases y los nombres. epochs=600: El modelo se entrenará durante 600 épocas (pasadas completas por el dataset). batch=256: Se procesan 256 imágenes a la vez en cada batch. imgsz=640: Todas las imágenes se redimensionan a 640×640 píxeles. scale=0.5: Este parámetro, junto con otros de augmentación, puede controlar la magnitud de ciertas transformaciones. mosaic=1.0: Habilita el mosaic augmentation (una técnica que combina partes de diferentes imágenes para formar una sola imagen de entrenamiento), muy utilizada en YOLO. mixup=0.0: Indica que no se utiliza mixup augmentation, aunque en otros modelos se podría activar con valores pequeños. copy_paste=0.1: Activa copy-paste augmentation con una probabilidad de 0.1 (esta técnica inserta partes de objetos de una imagen en otra). device="0,1,2,3": Especifica que se entrenará usando las GPUs con IDs 0, 1, 2 y 3. (En Colab normalmente se tiene una sola GPU, pero en otros entornos con múltiples GPUs esto se puede configurar).

# 6. Seguir proceso de entrenamiento

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolo12_resultados1

# 7. Guardar resultados del modelo

7.1 Para guardar los resultados!(moverlos de donde se guardan a donde quiero)


In [ ]:
# Crear la carpeta de destino

# !mkdir -p /content/mi_carpeta_resultados
!mkdir -p /content/yolo12_resultados1

# Copiar la carpeta de resultados (ajusta la ruta de origen según dónde se guarden tus resultados)

#normalmente se guardan en:
#/content/yolov12/runs/detect
#el código que quiero para copiarlos a la carpeta que quiero es:
#!cp -r /content/runs/detect/mi_experimento/ /content/mi_carpeta_resultados/

#cp -r /content/yolov12/runs/detect /content/yolo12_resultados1


##o si quiero moverlos en vez de copiarlos:

#!mv /content/runs/detect/mi_experimento/ /content/mi_carpeta_resultados/
!mv /content/yolov12/runs/detect /content/yolo12_resultados1


# Listar la carpeta de destino para verificar la copia
!ls -lh /content/yolo12_resultados1

total 4.0K
drwxr-xr-x 6 root root 4.0K Mar  6 16:02 detect


7.2 Guardar resultados a Drive

In [ ]:
# 10. Guardar los resultados del experimento

# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/Fotos_elegidas/resultados_yolo12"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/yolo12_resultados1 "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"                                                                 #verifica que los archivos se han copiado correctamente

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
#!zip -r /content/yolo12_resultados1.zip "{dest_folder}/focas_prueba_4_images"   #Aquí se comprime la carpeta que acabas de copiar en Drive (la subcarpeta focas_training_prueba1_640_24_02_25 dentro de foquitas) en un archivo ZIP llamado foquitas.zip que se guarda en /content (la carpeta principal del entorno de Colab).

# Descargar el archivo ZIP a tu PC
#from google.colab import files
#files.download('/content/foquitas.zip')

yolo12_resultados1


# 8. Validation

Ajustar IOU y confianza

In [ ]:
# 3. Evaluar el modelo en el conjunto de validación.

model.val(conf=0.25, iou=0.45)  #para cambiar dichos parametros en la comparación y así sacar datos como recall y precision

metrics = model.val()




9. Validación de su modelo inicial para ver si funciona contigo

esto es una validación si lo que quieres es probar unos archivos en unmodelo ya entrenado o en el modelo entrenado de ultralytics, no para tu vlidación

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov12{n/s/m/l/x}.pt')
model.val(data='coco.yaml', save_json=True)

inferencia de la validación del modelo

In [ ]:
# 4. Realizar inferencia en una imagen individual y mostrar el resultado.
results = model("path/to/image.jpg")
results[0].show()

# 10. Inferencia

Es la inferencia del modelo entrenado

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov12{n/s/m/l/x}.pt')
model.predict()

# 11. Export

Exportar el modelo en el formato adecuado para usarlo luego

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov12{n/s/m/l/x}.pt')
model.export(format="engine", half=True)  # or format="onnx"